In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# if latex not installed, set to False and remove tex symbols from plots
plt.rcParams["text.usetex"] = True

font = {"family": "normal", "weight": "bold", "size": 18}
import matplotlib

matplotlib.rc("font", **font)

from vgi import *
from supply_chain import *

# Create problem instance

In [ ]:
problem = SupplyChainProblem.create_problem_instance(processes=5)
V_lb = problem.V_lb()

# to create policies without compiling with cvxpygen, set compile=False
# create CE-MPC policy
mpc = problem.create_policy(lookahead=30, compile=True, name="supply_chain_mpc")

# create ADP policy
cocp = problem.create_policy(compile=True, name="supply_chain_policy", V=V_lb)

# Run VGI

In [ ]:
# run VGI
vgi = VGI(
    problem,
    cocp,
    QuadGradReg(),
    trajectory_len=50,
    num_trajectories=1,
    damping=0.5,
)
vgi_policy = vgi(10, V0=V_lb, eval_freq=1, seed=0)

# MPC

In [ ]:
mpc_cost = problem.cost(mpc, seed=1)
print("mpc cost: {:.2f}".format(mpc_cost))

# Plot VGI progress

In [ ]:
vgi_steps = [50 * i for i in range(len(vgi.costs))]
plt.figure(figsize=(8, 5))
plt.hlines(
    mpc_cost,
    0,
    vgi_steps[-1],
    label="CE-MPC",
    linestyles="dashed",
    color="orange",
    linewidth=4,
)
plt.step(vgi_steps, vgi.costs, label="VGI", linewidth=4)
plt.legend()
plt.grid()
plt.ylabel("Cost")
plt.xlabel("Policy evaluations")
plt.tight_layout()
plt.savefig("supply_chain_vgi_ce.pdf", dpi=500)
plt.show()
print("vgi", vgi.costs[-1], "steps", vgi_steps[-1])
print(vgi.costs[0], vgi.costs[-1], (vgi.costs[0] - vgi.costs[-1]) / vgi.costs[0])
print(vgi.costs[0], vgi.costs[-1], (mpc_cost - vgi.costs[-1]) / mpc_cost)

# Plot trajectory

In [ ]:
# initial policy
initial_cocp = cocp.clone()
initial_cocp.update_value(vgi.iterates[0])

# vgi policy
vgi_cocp = cocp.clone()
vgi_cocp.update_value(vgi.iterates[-1])# Save data

# plot trajectory averaged over 500 simulation
T = 50
runs = 500
untuned_states = np.zeros((T, problem.n))
vgi_states = np.zeros((T, problem.n))
for i in range(runs):
    traj_untuned = problem.simulate(initial_cocp, T, seed=i)
    traj_vgi = problem.simulate(vgi_cocp, T, seed=i)

    untuned_states += traj_untuned.states_matrix[:, : problem.n] / runs
    vgi_states += traj_vgi.states_matrix[:, : problem.n] / runs

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 3), sharey=True, dpi=500)

# Plot data in the first subplot
axs[0].plot(untuned_states[:, 0], label="$(h_t)_1$", linestyle="solid", linewidth=4)
axs[0].plot(untuned_states[:, 1], label="$(h_t)_2$", linestyle="dashed", linewidth=4)
axs[0].plot(untuned_states[:, 2], label="$(h_t)_3$", linestyle="dashdot", linewidth=4)
axs[0].plot(untuned_states[:, 3], label="$(h_t)_4$", linestyle="dotted", linewidth=4)
axs[0].set_title("Initial policy")
axs[0].set_xlabel("$t$")
axs[0].grid(True)
axs[0].set_ylabel("$h_t$")

axs[1].plot(vgi_states[:, 0], label="$(h_t)_1$", linestyle="solid", linewidth=4)
axs[1].plot(vgi_states[:, 1], label="$(h_t)_2$", linestyle="dashed", linewidth=4)
axs[1].plot(vgi_states[:, 2], label="$(h_t)_3$", linestyle="dashdot", linewidth=4)
axs[1].plot(vgi_states[:, 3], label="$(h_t)_4$", linestyle="dotted", linewidth=4)
axs[1].set_title("VGI policy")
axs[1].set_xlabel("$t$")
axs[1].grid(True)

# Create a legend for both plots
handles, labels = axs[1].get_legend_handles_labels()
fig.legend(handles, labels, loc="center left", bbox_to_anchor=(0.9, 0.5))

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.1)
plt.savefig("supply_chain_trajectories.pdf", dpi=500, bbox_inches="tight")
plt.show()

# Fitted value iteration

In [ ]:
fvi = FVI(
    problem,
    cocp,
    QuadReg(l2_penalty=1e-4),
    trajectory_len=400,
    num_trajectories=2,
    damping=0.75,
    restart_simulations=True
)
fvi_policy = fvi(25, V0=V_lb, eval_freq=1, seed=1)

# COCP gradient

In [ ]:
# run cocp gradient
trajectory_len = 100
num_trajectories = 10
num_iters = 100
learning_rate = 1e-2
cocp_grad = supply_chain_cocp_grad(
    problem,
    trajectory_len,
    num_iters,
    learning_rate,
    seed=2,
    l2_penalty=1e-4,
    V0=V_lb,
    num_trajectories=num_trajectories,
    policy=cocp,
    eval_freq=5,
    restart_simulations=True,
)

# Save data

In [ ]:
import pickle

results = {
    "vgi": {"costs": vgi.costs, "iterates": vgi.iterates},
    "fvi": {"costs": fvi.costs, "iterates": fvi.iterates},
    "cocp-grad": cocp_grad,
}
pickle.dump(results, open("supply_chain_results.pkl", "wb"))